# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
# from line_profiler import LineProfiler

%matplotlib inline

# %load_ext pycodestyle_magic


In [4]:
def profile_print(func_to_call, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()
    

In [71]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.simple_G_function = self.__simple_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.simple_G_function = self.__simple_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.simple_G_function = self.__simple_misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __simple_gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        return y.size * (1 - np.sum((counts / y.size) ** 2))

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype(np.float)
        r_s = r_s.astype(np.float)
        return l_s * (1 - np.sum((l_c / l_s) ** 2, axis=1)[:, np.newaxis]) + \
            r_s * (1 - np.sum((r_c / r_s) ** 2, axis=1)[:, np.newaxis])

    def __simple_entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        return -y.size * np.sum((counts / y.size) * np.log2(counts / y.size))

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype(np.float)
        r_s = r_s.astype(np.float)
        return -l_s * np.sum(l_c / l_s * np.log2(l_c / l_s),
                             axis=1)[:, np.newaxis] - r_s * np.sum(
            r_c / r_s * np.log2(r_c / r_s), axis=1)[:, np.newaxis]

    def __simple_misclass(self, y):
        _, counts = np.unique(y, return_counts=True)
        return y.size * (1 - np.max(counts / y.size))

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype(np.float)
        r_s = r_s.astype(np.float)
        return l_s * (1 - np.max(l_c / l_s, axis=1)[:, np.newaxis]) + \
            r_s * (1 - np.max(r_c / r_s, axis=1)[:, np.newaxis])

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return features_ids[:np.sqrt(n_features)],
        features_ids[:-np.sqrt(n_features)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return features_ids[:np.log2(n_features)],
        features_ids[:-np.log2(n_features)]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature), []

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        eq_mask = x[:, feature_id] == threshold
        right_mask = x[:, feature_id] < threshold
        x_l = x[left_mask]
        x_r = x[right_mask]
        y_l = y[left_mask]
        y_r = y[right_mask]
        if y_l.size == 0:
            if y_r.size == 0:
                x_l = x[0, :]
                x_r = x[1:, :]
                y_l = y[0]
                y_r = y[1:]
            else:
                y_l = y[eq_mask]
                x_l = x[eq_mask]
        else:
            x_r = x[right_mask | eq_mask]
            y_r = y[right_mask | eq_mask]
        return x_l, x_r, y_l, y_r

    def __find_threshold(self, x, y):
        # возвращает отсортированный по признакам х и соответствующие ему у,
        # количество уникальных элементов в у, т.е. кол-во классов 
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # находит индексы, на которых происходит
        # изменение в у с учетом min_samples_split, и отбрасывает случай,
        # когда таких нет
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # сначала получаем массив кол-ва одинаковых эл-тов у; 
        # делаем one_hot_code:
        # переходим к матрице 0 и 1, и делаем матрицу количеств каждого
        # класса в группках r_border_ids
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Считаем сколько семплов попадут в правый и в левый потомок
        # при этом строчки отвечают за индекс разбиения (через r_border_ids)
        # а столбцы за значение целевой переменной
        # количество семплов в левом и правом потомке для каждого разбиения
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # находим impurity для всех разбиений
        # получаем индекс разбиения с минимальным impurity
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # получаем индекс 'крайнего' элемента, который удовлетворяет
        # выбранному разбиению и возвращаем оптимальные значения
        # признаков для рабиения и значение неопределенности
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        (classes, counts) = np.unique(y, return_counts=True)
        cl_ind = np.argmax(counts)
        cl = classes[cl_ind]
        if depth == self.max_depth or counts[cl_ind] / y.size >= \
                self.sufficient_share:
            self.tree[node_id] = (self.LEAF_TYPE, cl)
            return
        feature_ids, other_features = self.get_feature_ids(x.shape[1])
        gs_threshold = np.full((2, x.shape[1]), np.inf)
        gs_threshold[:, feature_ids] = np.apply_along_axis(
            self.__find_threshold, 0, x[:, feature_ids], y)
        gs_threshold[0, pred_f] = np.inf
        if gs_threshold[0, :] is np.full((x.shape[1]), np.inf):
            for feature in other_features:
                if feature == pred_f:
                    continue
                gs_threshold[0, feature], gs_threshold[1, feature] = \
                    self.__find_threshold(x[:, feature], y)
                if gs_threshold[1, feature] is None:
                    continue
                break
        if gs_threshold[0, :] is np.full((x.shape[1]), np.inf):
            self.tree[node_id] = (self.LEAF_TYPE, cl)
            return
        feature = np.argmin(gs_threshold[0, :])
        x_l, x_r, y_l, y_r = self.__div_samples(
            x, y, feature, gs_threshold[1, feature])
        if y_l.size == 0 or y_r.size == 0:
            print('broken')
            return
        self.tree[node_id] = (self.NON_LEAF_TYPE,
                              feature, gs_threshold[1, feature])
        self.feature_importances_[feature] += self.simple_G_function(y) - \
            self.simple_G_function(y_l) - self.simple_G_function(y_r)
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, feature)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, feature)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    

In [72]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)


In [73]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)


## Проверка скорости работы на wine

In [74]:
%time clf.fit(X_train, y_train)


CPU times: user 1.83 ms, sys: 16 µs, total: 1.84 ms
Wall time: 1.66 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [75]:
%time my_clf.fit(X_train, y_train)


CPU times: user 9.26 ms, sys: 3.94 ms, total: 13.2 ms
Wall time: 13.2 ms


## Проверка качества работы на wine

In [79]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')


0.7833333333333333

In [77]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')


0.882051282051282

## Подготовка данных Speed Dating Data 

In [80]:
# %%pycodestyle

df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
              'order', 'partner', 'age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'field',
              'undergra', 'from', 'zipcode', 'career',
              'sports', 'tvsports', 'exercise', 'dining', 'museums',
              'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies',
              'concerts', 'music', 'shopping', 'yoga',
              'expnum', 'wave'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(
    ',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(
    ',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(
    ',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1',
                                        'intel1_1', 'fun1_1',
                                        'amb1_1', 'shar1_1']].sum(
    axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
           'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                'amb1_1', 'shar1_1']].T/df.loc[
        :, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(
    axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
           'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                'amb2_1', 'shar2_1']].T/df.loc[
        :, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=[
    'iid', 'pid']).drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(
    subset=['iid']).drop(['gender', 'match',
                          'int_corr', 'samerace'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values


In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)


In [82]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)


## Проверка скорости работы на Speed Dating Data 

In [83]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
%time clf.fit(X_train, y_train)


CPU times: user 67 ms, sys: 0 ns, total: 67 ms
Wall time: 66.6 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [84]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
%time my_clf.fit(X_train, y_train)


CPU times: user 4.04 s, sys: 0 ns, total: 4.04 s
Wall time: 4.04 s


## Проверка качества работы на Speed Dating Data

In [85]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')


0.562624702686251

In [86]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')


0.5407577497129735

## Задание 3

In [87]:
pipeline = Pipeline([
    ('clf', DecisionTreeClassifier()),
])

pipeline.fit(X_train, y_train)
f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro')


0.6061475234385033

## Задание 4

In [88]:
model1 = DecisionTreeClassifier()
model1.fit(X_train, y_train)

model2 = MyDecisionTreeClassifier()
model2.fit(X_train, y_train)


In [89]:
features_names = df_pair.columns[1:]


In [90]:
print('DecisionTreeClassifier')
print(pd.Series(index=features_names, data=model1.feature_importances_)\
.sort_values(ascending=False).head(10))
print('MyDecisionTreeClassifier')
print(pd.Series(index=features_names, data=model2.feature_importances_)\
.sort_values(ascending=False).head(10))


DecisionTreeClassifier
int_corr     0.090493
fun2_1       0.032057
attr1_1_f    0.029492
fun1_1       0.029010
date         0.027105
shar1_1      0.026008
age_f        0.025757
age          0.025551
fun2_1_f     0.025150
income_f     0.023032
dtype: float64
MyDecisionTreeClassifier
int_corr      0.062116
age           0.017466
samerace      0.017017
age_f         0.009525
field_cd_f    0.007767
imprace       0.007088
field_cd      0.006475
attr3_1_f     0.006360
attr1_1       0.005209
mn_sat_f      0.005193
dtype: float64


In [91]:
print(f1_score(y_pred=model1.predict(X_test), y_true=y_test, average='macro'))
print(f1_score(y_pred=model2.predict(X_test), y_true=y_test, average='macro'))


0.5757575757575757
0.5407577497129735


## Задание 5

In [94]:
# Speed Dating Data
param_dist = {"max_depth": [1, 2, 3, 4, 5, 6, 7, 8, None],
              "min_samples_split": range(2, 11),
              "criterion": ["gini", "entropy"]}

gscv = RandomizedSearchCV(RandomForestClassifier(n_estimators=10), param_dist, n_iter=10)
gscv.fit(X_train, y_train)
print(gscv.best_params_)


{'min_samples_split': 5, 'max_depth': 8, 'criterion': 'gini'}
